# 14 - PointNet++ Feature Propagation Layer (Code Study)

This notebook is a detailed code study of the PointNet++ Feature Propagation (FP) layer.

It is based on the Keras layer implememtation of PointNet++ from https://github.com/dgriffiths3/pointnet2-tensorflow2.

Follow the notebook to better understand the Feature Propagation layer and its implementation. It is assumed that you already completed the code study of the Set Abstraction layer. Therefore, repeated sections are not explained once more.

## Setup TensorFlow

In [1]:
# Change X to the GPU number you want to use,
# otherwise you will get a Python error
# e.g. USE_GPU = 4
USE_GPU = 4

In [2]:
# Import TensorFlow 
import tensorflow as tf

# Print the installed TensorFlow version
print(f'TensorFlow version: {tf.__version__}\n')

# Get all GPU devices on this server
gpu_devices = tf.config.list_physical_devices('GPU')

# Print the name and the type of all GPU devices
print('Available GPU Devices:')
for gpu in gpu_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set only the GPU specified as USE_GPU to be visible
tf.config.set_visible_devices(gpu_devices[USE_GPU], 'GPU')

# Get all visible GPU  devices on this server
visible_devices = tf.config.get_visible_devices('GPU')

# Print the name and the type of all visible GPU devices
print('\nVisible GPU Devices:')
for gpu in visible_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set the visible device(s) to not allocate all available memory at once,
# but rather let the memory grow whenever needed
for gpu in visible_devices:
    tf.config.experimental.set_memory_growth(gpu, True)
    
# Import Keras
from tensorflow import keras

# Print the installed Keras version
print(f'\nKeras version: {keras.__version__}\n')

TensorFlow version: 2.3.1

Available GPU Devices:
  /physical_device:GPU:0 GPU
  /physical_device:GPU:1 GPU
  /physical_device:GPU:2 GPU
  /physical_device:GPU:3 GPU
  /physical_device:GPU:4 GPU
  /physical_device:GPU:5 GPU
  /physical_device:GPU:6 GPU
  /physical_device:GPU:7 GPU

Visible GPU Devices:
  /physical_device:GPU:4 GPU

Keras version: 2.4.0



## Prepare TensorFlow CUDA operations

In this section, the TensorFlow operations implemented in CUDA (a programming language for GPUs from NVIDIA) are prepared to be used in the PointNet++ Jupyter notebooks.

**Note that this section of the notebook needs only be executed once to install and compile the TensorFlow CUDA operations.** But there should be no harm in executing it repeatedly. 

**Make sure that the file 'tf_ops.zip' is in the same folder as this notebook and all other notebooks you want to use these TensorFlow operations with.**

The following is commented out, assuming you already did this. If not, then uncomment and execute.

In [3]:
#!unzip -o -q "tf_ops.zip"

#!chmod u+x "tf_ops/compile_ops.sh"

#!tf_ops/compile_ops.sh

## Load a point cloud patch

In [4]:
import numpy as np
import pandas as pd

In [5]:
from pathlib import Path
import os

# directory of PointNet data
data_dir = str(Path.home()) + r'/coursematerial/GIS/ISPRS/PointNet++'

# filename and path of one patch
filename = r'Vaihingen3D_Training_0016.csv'
filepath = os.path.join(data_dir, 'patches', filename)

# read csv file as Pandas DataFrame
xyz_df = pd.read_csv(filepath, sep=' ')

# extract x,y,z-columns from DataFrame and convert to NumPy array
xyz = xyz_df[['x','y','z']].to_numpy()

# center by center point of bounding box
min = np.min(xyz, axis=0)
max = np.max(xyz, axis=0)
xyz = xyz - np.expand_dims(0.5 * (max+min), axis=0)

# extract the labels from DataFrame
labels = xyz_df[['labels']].to_numpy()

In [6]:
xyz = tf.convert_to_tensor(np.expand_dims(xyz, axis=0), dtype=tf.float32)

xyz

<tf.Tensor: shape=(1, 100000, 3), dtype=float32, numpy=
array([[[ 1.0000e-02, -8.5000e-02, -1.8950e+00],
        [ 1.0000e-02,  1.8500e-01, -1.8450e+00],
        [ 1.0000e-02,  1.9500e-01, -1.8350e+00],
        ...,
        [ 4.4900e+01, -3.2345e+01, -4.9250e+00],
        [-5.2490e+01, -1.7135e+01, -3.6250e+00],
        [-5.2490e+01, -1.7135e+01, -3.6250e+00]]], dtype=float32)>

## Interpolation

### Extract 4k & 1K points

The Feature Propagation layer takes two point sets as input, one with fewer points and one with more points. We therefore use farthest point sampling to sample first a tensor of 4096 points with xyz-coordinates and from this one another tensor of 1024 points with xyz-coordinates.

In [7]:
# import tf operations
from tf_ops.tf_ops import (
    farthest_point_sample,
    three_nn,
    three_interpolate
)

In [8]:
xyz4096 = tf.gather(xyz, farthest_point_sample(4096, xyz), axis=1, batch_dims=1)
print(xyz4096.shape)

xyz1024 = tf.gather(xyz4096, farthest_point_sample(1024, xyz4096), axis=1, batch_dims=1)
print(xyz1024.shape)

(1, 4096, 3)
(1, 1024, 3)


And construct some feature tensors of random values with 5 and 8 channels. The point set with fewer points has more features. (Typically many more than just 5 and 8.)

In [9]:
fea4096 = tf.random.uniform(shape=[1, 4096, 5])
fea1024 = tf.random.uniform(shape=[1, 1024, 8])

### Indices of 3 nearest neighbors

As we want to propagate the features from the point set with fewer points to the point set with more points, we have to identify the points from which to take the features as input.

The function **three_nn()** determines for each point of the set of 4096 points the indices of the 3 nearest neighbor points of the set containing 1024 points. 

In [10]:
dist, idx = three_nn(xyz4096, xyz1024)
print(dist.shape)
print(idx.shape)

(1, 4096, 3)
(1, 4096, 3)


In [11]:
# 3 indices of first point (of first batch)
idx[0, 0]

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  0, 944, 732], dtype=int32)>

Notice that the point includes itself as one of the 3 nearest neighbors. Points like this are points that are in both sets. (Which is not surprising for the first point, as the first point is always sampled in farthest point sampling.) Therefore, we choose another point at index 2005.

In [12]:
# 3 indices of 2006th point (of first batch)
idx[0, 2005]

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([521, 419, 205], dtype=int32)>

In [13]:
dist[0, 2005]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 6.4746046,  8.125095 , 13.816187 ], dtype=float32)>

### Weights for inverse distance weighting (IDW)

We now calculate the weights for the inverse distance weighting. This is a vector implementation that calculates the weights for the complete set of 4096 points. (If you do not know inverse distance weighting, then please check https://en.wikipedia.org/wiki/Inverse_distance_weighting. The idea is basically that the weights are inversely proportional to the distances, where the weights must sum up to 1. Therefore, the weights are normalized, here by dividing by norm.)

In [14]:
# make sure there is no distance of 0.0, as we need to divide by the distance
dist = tf.maximum(dist, 1e-10)

# sum the 3 distances per point
norm = tf.reduce_sum((1.0/dist),axis=2, keepdims=True)

# repeat the sum for all three distances
norm = tf.tile(norm,[1,1,3])

# divide and normalize
weight = (1.0/dist) / norm

print(weight.shape)

(1, 4096, 3)


The weights for points that are in both sets (4096 and 1024) is calculated as 1.0, so that it keeps its features.

In [15]:
weight[0, 0]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.000000e+00, 9.407515e-12, 6.689276e-12], dtype=float32)>

For all other points, the weights are a combination of the 3 points.

In [16]:
weight[0, 2005]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.44140565, 0.35174075, 0.20685355], dtype=float32)>

### Interpolate with IDW

Using the indices and the calculated weights, the CUDA operation **three_interpolate()** performs the interpolation.

In [17]:
interpolated_points = three_interpolate(fea1024, idx, weight)
print(interpolated_points.shape)

(1, 4096, 8)


As you notice from the outputs of the following cell, the features of the first point that appears in both point sets keep their values.

In [18]:
print(fea1024[0, 0])
print(interpolated_points[0, 0])

tf.Tensor(
[0.05276966 0.41259992 0.96929705 0.8000699  0.45917094 0.20513296
 0.70916283 0.562122  ], shape=(8,), dtype=float32)
tf.Tensor(
[0.05276966 0.41259992 0.96929705 0.8000699  0.45917094 0.20513296
 0.70916283 0.562122  ], shape=(8,), dtype=float32)


While the feature values of the point at index 2005 is the interpolation of the 3 nearest neighbors.

In [19]:
print('Input tensors:')
print(fea1024[0, idx[0, 2005, 0]])
print(fea1024[0, idx[0, 2005, 1]])
print(fea1024[0, idx[0, 2005, 2]])

print('\nWeights:')
print(weight[0, 2005])

print('\nInterpolated tensor:')
print(interpolated_points[0, 2005])

Input tensors:
tf.Tensor(
[0.6469661  0.7789264  0.5930282  0.7617823  0.90258455 0.23327053
 0.1712929  0.04030871], shape=(8,), dtype=float32)
tf.Tensor(
[0.04414821 0.85745466 0.39262962 0.39014578 0.70860076 0.2864374
 0.43942547 0.33796513], shape=(8,), dtype=float32)
tf.Tensor(
[0.43107855 0.53848195 0.6230464  0.5000411  0.9263269  0.6379752
 0.6030979  0.40158522], shape=(8,), dtype=float32)

Weights:
tf.Tensor([0.44140565 0.35174075 0.20685355], shape=(3,), dtype=float32)

Interpolated tensor:
tf.Tensor(
[0.39027336 0.75681114 0.5287492  0.57692045 0.8392637  0.3356861
 0.35492644 0.21973795], shape=(8,), dtype=float32)


### Combine features from both layers

At the end, the interpolated features and point features (that are with the 4096 points from the set abstraction layer) are concatenated.

In [20]:
new_fea4096 = tf.concat(axis=2, values=[interpolated_points, fea4096]) # B,ndataset1,nchannel1+nchannel2
print(new_fea4096.shape)

(1, 4096, 13)


### Multi-layer perceptron

What follows in the Feature Propagation layer is the multi-layer perceptron that is applied on the concatenated features.

Because the 2D convolutional layers require 4D tensors, the dimension of the feature tensor needs to be expanded first.

In [21]:
new_fea4096_xdim = tf.expand_dims(new_fea4096, 2)
print(new_fea4096_xdim.shape)

(1, 4096, 1, 13)


At this point, the concatenated feature tensor goes through a multi-layer perceptron. As we do not have a trained MLP in this notebook, the following cell is commented out.

The list mlp_list contains 2D convolutional layers. The for-loop iterates over all these layers, and applies the tensor to every layer. The resulting tensor has the same dimensions with the exception of the last dimension, which has as many channels as the last convolutional layer has filters.

We just generate a tensor with random values with an appropriate shape.

In [22]:
#for i, mlp_layer in enumerate(self.mlp_list):
#    new_points1 = mlp_layer(new_points1, training=training)

mlp_output = tf.random.uniform(shape=[1, 4096, 1, 128])

print(mlp_output.shape)

(1, 4096, 1, 128)


### Reshape output tensor

The following cell first gets rid of the dimensions with size 1 and re-introduced a batch dimension (at axis 0) once more.

In [23]:
mlp_output_reshaped = tf.squeeze(mlp_output)
print('Squeezed:', mlp_output_reshaped.shape)

if len(mlp_output_reshaped.shape) < 3:
    mlp_output_reshaped = tf.expand_dims(mlp_output_reshaped, axis=0)

print('Expanded:', mlp_output_reshaped.shape)

Squeezed: (4096, 128)
Expanded: (1, 4096, 128)


## Feature Propagation (FP) layer

In the following, a (simplified) implementation of the Feature Propagation layer is given as a TensorFlow custom layer.

It follows to a large extent what is discussed above. There is some code at the beginning of **call()** that makes sure that the dimensions of the input features are as expected and expands them if they are not.

Note the following parameters:

- xyz1 is the re-introduced point set (skip connection)
- xyz2 is the point set from the previous layer
- points1 are the features associated with xyz1
- points2 are the features associated with xyz2

Note that the feature tensor of the point set with the target points might be empty for the last feature propagation layer, since this is the layer that gets the points from the first set abstraction layer (via skip connection). If the network did not receive any input features, then the skip connections does not provide any features as well. Therefore, points1 could be None and the layer must handle this case by not concatenating points1 with the interpolated points. Then the output is just the interpolated point features.

In [24]:
from tensorflow.keras.layers import Layer

class Pointnet_FP(Layer):

    def __init__(self, mlp):
        super(Pointnet_FP, self).__init__()

        self.mlp = mlp
        self.mlp_list = []


    def build(self, input_shape):

        for i, n_filters in enumerate(self.mlp):
            self.mlp_list.append(keras.layers.Conv2D(n_filters, kernel_size=[1,1], activation='relu'))

        super(Pointnet_FP, self).build(input_shape)

    def call(self, xyz1, xyz2, points1, points2, training=True):

        if points1 is not None:
            if len(points1.shape) < 3:
                points1 = tf.expand_dims(points1, axis=0)
        if points2 is not None:
            if len(points2.shape) < 3:
                points2 = tf.expand_dims(points2, axis=0)

        dist, idx = three_nn(xyz1, xyz2)
        dist = tf.maximum(dist, 1e-10)
        norm = tf.reduce_sum((1.0/dist),axis=2, keepdims=True)
        norm = tf.tile(norm,[1,1,3])
        weight = (1.0/dist) / norm
        interpolated_points = three_interpolate(points2, idx, weight)

        if points1 is not None:
            new_points1 = tf.concat(axis=2, values=[interpolated_points, points1]) # B,ndataset1,nchannel1+nchannel2
        else:
            new_points1 = interpolated_points
        new_points1 = tf.expand_dims(new_points1, 2)

        for i, mlp_layer in enumerate(self.mlp_list):
            new_points1 = mlp_layer(new_points1, training=training)

        new_points1 = tf.squeeze(new_points1)
        if len(new_points1.shape) < 3:
            new_points1 = tf.expand_dims(new_points1, axis=0)

        return new_points1

Please continue with the PointNet++ exercise.